# Electronic Market E-Commerce Data Analysis

## 1. Business Context & Goals

Electronic Market is a digital-first e-commerce platform focused on consumer electronics. Operating since 2019, it faced a significant revenue boom during the COVID-19 pandemic, followed by a decline as physical retail rebounded in 2021. This analysis aims to uncover patterns in revenue, customer behavior, product performance, and regional dynamics to deliver actionable recommendations for business optimization.

## 2. Data Loading & Preview

## Executive KPI Summary

The following table summarizes key business metrics extracted from the Electronic Market dataset:

In [ ]:
import pandas as pd

# Precomputed KPI values
summary_df = pd.DataFrame({
    "Metric": [
        "Total Revenue (2021)",
        "Repeat Buyer Rate",
        "Top Product Revenue %",
        "Avg. Time to Ship (Corrected)",
        "Regional Leader"
    ],
    "Value": [
        "$1,240,539",
        "22.5%",
        "12.8%",
        "2.0 days",
        "EMEA / 36.4%"
    ]
})

display(summary_df)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-deep')
sns.set_theme()

# Load Excel file
xls = pd.ExcelFile("electronic_market_dataset.xlsx")
orders_raw = xls.parse("orders_cleaned")
orders = orders_raw[1:].copy()
orders.columns = orders_raw.iloc[0]
orders = orders.dropna(subset=["USD_PRICE"])

# Preview loaded data
display(orders.head())

## 3. Data Cleaning & Preparation

In [ ]:
# Type casting and formatting
orders["USD_PRICE"] = orders["USD_PRICE"].astype(float)
orders["PURCHASE_TS_CLEANED"] = pd.to_datetime(orders["PURCHASE_TS_CLEANED"], errors='coerce')
orders["PURCHASE_YEAR"] = pd.to_numeric(orders["PURCHASE_YEAR"], errors='coerce').astype("Int64")
orders["PURCHASE_MONTH"] = pd.to_numeric(orders["PURCHASE_MONTH"], errors='coerce').astype("Int64")
orders = orders.dropna(subset=["PURCHASE_YEAR", "PURCHASE_MONTH"])

# Clean marketing channels
orders["MARKETING_CHANNEL_CLEANED"] = orders["MARKETING_CHANNEL_CLEANED"].str.lower()

# Extract Year-Month
orders["Year-Month"] = orders["PURCHASE_TS_CLEANED"].dt.to_period("M").astype(str)

# Quick summary
print(orders.describe(include='all'))

## 4. Exploratory Data Analysis (EDA)
### Monthly Sales Trend

In [ ]:
monthly_revenue = orders.groupby("Year-Month")["USD_PRICE"].sum().reset_index()
plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_revenue, x="Year-Month", y="USD_PRICE", marker="o")
plt.title("Monthly Sales Trend (USD)")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

*Dashboard: Monthly revenue patterns with seasonal peaks and slumps visualized.*

### Annual Growth Rates

In [ ]:
annual_revenue = orders.groupby("PURCHASE_YEAR")["USD_PRICE"].sum().reset_index()
annual_revenue["Growth Rate (%)"] = annual_revenue["USD_PRICE"].pct_change() * 100
plt.figure(figsize=(8, 5))
sns.barplot(data=annual_revenue, x="PURCHASE_YEAR", y="Growth Rate (%)", palette="gray")
plt.title("Annual Revenue Growth Rate")
plt.grid(axis='y')
plt.tight_layout()
plt.show()

*Dashboard: Percentage growth visualization year-over-year to assess volatility.*

## 5. Regional Sales Insights

In [ ]:
regional_sales = orders.groupby("REGION")["USD_PRICE"].sum().reset_index().sort_values(by="USD_PRICE", ascending=False)
plt.figure(figsize=(8, 5))
sns.barplot(data=regional_sales, x="USD_PRICE", y="REGION", palette="gray")
plt.title("Sales by Region")
plt.xlabel("Total Revenue ($)")
plt.tight_layout()
plt.show()

*Dashboard: Regional contribution comparison to guide geo-targeting efforts.*

## 6. Platform & Marketing Channel Distribution

In [ ]:
# Platform
platform_stats = orders["PURCHASE_PLATFORM"].value_counts().reset_index()
platform_stats.columns = ["Platform", "Number of Orders"]

plt.figure(figsize=(6, 5))
sns.barplot(data=platform_stats, x="Number of Orders", y="Platform", palette="gray")
plt.title("Sales by Platform")
plt.tight_layout()
plt.grid(axis="x")
plt.show()

*Dashboard: Platform performance revealing desktop vs mobile dominance.*

In [ ]:
# Marketing Channel
channel_stats = orders["MARKETING_CHANNEL_CLEANED"].value_counts().reset_index()
channel_stats.columns = ["Marketing Channel", "Number of Orders"]

plt.figure(figsize=(6, 5))
sns.barplot(data=channel_stats, x="Number of Orders", y="Marketing Channel", palette="gray")
plt.title("Sales by Marketing Channel")
plt.tight_layout()
plt.grid(axis="x")
plt.show()

*Dashboard: Marketing effectiveness by channel, highlighting overdependence.*

## 7. Customer Behavior Analysis
### Repeat vs. One-Time Buyers

In [ ]:
repeat_customers = orders.groupby("USER_ID")["ORDER_ID"].count().reset_index(name="Order Count")
repeat_customers["Type"] = repeat_customers["Order Count"].apply(lambda x: "Repeat Buyer" if x > 1 else "One-Time Buyer")
repeat_summary = repeat_customers["Type"].value_counts().reset_index()
repeat_summary.columns = ["Type", "Number of Customers"]

plt.figure(figsize=(6, 5))
sns.barplot(data=repeat_summary, x="Type", y="Number of Customers", palette="gray")
plt.title("Customer Type Distribution")
plt.tight_layout()
plt.grid(axis='y')
plt.show()

*Dashboard: Retention funnel view — clear need for loyalty programs.*

### Yearly Customer Growth

In [ ]:
customer_growth = orders.groupby("PURCHASE_YEAR")["USER_ID"].nunique().reset_index()
customer_growth.columns = ["Year", "New Customers"]

plt.figure(figsize=(8, 5))
sns.lineplot(data=customer_growth, x="Year", y="New Customers", marker="o")
plt.title("Yearly Customer Growth")
plt.grid(True)
plt.tight_layout()
plt.show()

*Dashboard: Acquisition trend indicating momentum loss post-pandemic.*

## 8. Product-Level Insights

In [ ]:
top_products = orders.groupby("PRODUCT_NAME_CLEANED")["USD_PRICE"].agg(['sum', 'count']).reset_index()
top_products.columns = ["Product", "Total Revenue", "Order Count"]
top_products = top_products.sort_values(by="Total Revenue", ascending=False).head(6)

plt.figure(figsize=(10, 6))
sns.barplot(data=top_products, x="Total Revenue", y="Product", palette="gray")
plt.title("Top Product Revenue Contribution")
plt.tight_layout()
plt.grid(axis='x')
plt.show()

## 9. Strategic Business Recommendations

- **Diversify Product Portfolio**: Reduce dependency on top 3 items by bundling and launching mid-tier accessories.
- **Improve Mobile Experience**: Drive engagement and sales via mobile-first improvements.
- **Customer Retention**: Launch loyalty tiers and personalized post-purchase offers.
- **Regional Activation**: Invest in localized marketing strategies for APAC and LATAM.
- **Channel Expansion**: Introduce social/email campaigns to reduce reliance on direct traffic.

## 10. Limitations and Caveats

While this analysis provides actionable insights, it is important to note the following limitations of the dataset and assumptions made:

- **No Returns Data**: The dataset does not include information about product returns, refunds, or cancellations.
- **No Cost Data**: There is no record of cost of goods sold (COGS), marketing spend, or shipping cost, limiting profitability analysis.
- **No Marketing Attribution**: Marketing channels are present but lack campaign identifiers or spend data.
- **No User Demographics**: Customer profiles do not include location, age, gender, or device metadata.
- **Assumes All Orders Were Fulfilled**: No data on failed, late, or incomplete orders.
- **Static Snapshot**: The dataset appears to be historical and does not reflect real-time or continuous data updates.
- **Missing Data**: Some fields have missing or inconsistent entries (e.g., time to ship, platform).

These caveats should be considered when interpreting trends and making decisions based on this analysis.